## Imports

In [1]:
import pandas as pd
import re
import ast
from collections import Counter
import itertools

## Dataset Overview

In [2]:
df = pd.read_csv('dataset.csv')
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


In [3]:
df.describe().T

,count,unique,top,freq
movie title,24402,23922,Rage,4
Run Time,24402,1556,not-released,8475
Rating,24402,91,no-rating,1740
User Rating,24402,1684,0,1740
Generes,24402,746,['Drama'],943
Overview,24158,23957,none,142
Plot Kyeword,24402,21546,[],1696
Director,24402,11604,See company contact information,142
Top 5 Casts,24402,24211,"['See producer', 'See preliminary cast']",142
Writer,24402,15562,See writer,142


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24402 entries, 0 to 24401
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie title   24402 non-null  object
 1   Run Time      24402 non-null  object
 2   Rating        24402 non-null  object
 3   User Rating   24402 non-null  object
 4   Generes       24402 non-null  object
 5   Overview      24158 non-null  object
 6   Plot Kyeword  24402 non-null  object
 7   Director      24402 non-null  object
 8   Top 5 Casts   24402 non-null  object
 9   Writer        24402 non-null  object
 10  year          23624 non-null  object
 11  path          24402 non-null  object
dtypes: object(12)
memory usage: 2.2+ MB


In [5]:
df.shape

(24402, 12)

In [6]:
df.isnull().sum()

movie title       0
Run Time          0
Rating            0
User Rating       0
Generes           0
Overview        244
Plot Kyeword      0
Director          0
Top 5 Casts       0
Writer            0
year            778
path              0
dtype: int64

In [7]:
# Fixing typo
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.rename(columns={'plot_kyeword': 'plot_keyword', 'generes': 'genres'})

## Dropping Unnecessary columns

In [8]:
# We are not using run time, year, and path to recommend, we should drop the columns.
df = df.drop('run_time', axis=1)
df = df.drop('year', axis=1)
df = df.drop('path', axis=1)
df = df.drop('overview', axis=1)

## Rating and Votes Parsing

In [9]:
# Change rating to numbers
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [10]:
def parse_votes(value):
    if isinstance(value, str):
        value = value.strip().upper().replace('K', '000').replace('M', '000000')
    try:
        return int(float(value))
    except:
        return None

df['votes'] = df['user_rating'].apply(parse_votes)

## Strings to Lists

In [11]:
for col in ['genres', 'plot_keyword', 'top_5_casts']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

## High quality Filtering

In [12]:
high_quality_df = df[(df['rating'] >= 6.0) & (df['votes'] >= 10000)]

## Genre Cleaning

In [13]:
# Genres Observation
genre_counter = Counter(itertools.chain.from_iterable(high_quality_df['genres']))
print(genre_counter.most_common())

[('Drama', 3663), ('Comedy', 2060), ('Action', 1418), ('Crime', 1330), ('Romance', 1020), ('Adventure', 991), ('Thriller', 954), ('Mystery', 574), ('Biography', 520), ('Horror', 516), ('Sci-Fi', 396), ('Fantasy', 380), ('Animation', 253), ('History', 235), ('Family', 223), ('Music', 180), ('War', 141), ('Sport', 128), ('Western', 91), ('Musical', 88), ('Film-Noir', 45)]


### One-Hot Encoding

In [14]:
# One-Hot Encode Genres
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(high_quality_df['genres'])

genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

high_quality_df = pd.concat([high_quality_df.reset_index(drop=True), genre_df], axis=1)

## Keywords Cleaning

In [15]:
# Keyword Observation
def normalize_keywords(keywords):
    keywords = [kw.lower().strip() for kw in keywords if isinstance(kw, str)]
    return sorted(set(keywords))

high_quality_df['plot_keyword'] = high_quality_df['plot_keyword'].apply(normalize_keywords)

keyword_counter = Counter(itertools.chain.from_iterable(high_quality_df['plot_keyword']))
print(keyword_counter.most_common())

[('murder', 257), ('female nudity', 217), ('based on novel', 198), ('new york city', 194), ('female protagonist', 177), ('based on true story', 171), ('friendship', 169), ('revenge', 163), ('police', 160), ('violence', 147), ('sex scene', 146), ('character name as title', 140), ('love', 134), ('dog', 123), ('father son relationship', 122), ('death', 121), ('husband wife relationship', 119), ('neo noir', 114), ('father daughter relationship', 111), ('female frontal nudity', 110), ('f rated', 110), ('cult film', 109), ('sequel', 105), ('prison', 103), ('marriage', 95), ('male nudity', 95), ('mother son relationship', 94), ('black comedy', 93), ('female rear nudity', 93), ('gangster', 88), ('small town', 88), ('high school', 84), ('bare chested male', 83), ('nudity', 83), ('kidnapping', 82), ('one word title', 82), ('psychotronic film', 81), ('mother daughter relationship', 79), ('police officer', 79), ('rape', 77), ('infidelity', 75), ('female full frontal nudity', 74), ('suicide', 74), 

### Filtering Out Low Frequency Keywords

In [16]:
threshold = 3
low_freq_keywords = {kw for kw, count in keyword_counter.items() if count <= threshold}
len(low_freq_keywords)

15294

In [17]:
def remove_low_freq_keywords(keywords):
    return [kw for kw in keywords if kw not in low_freq_keywords]

high_quality_df['plot_keyword'] = high_quality_df['plot_keyword'].apply(remove_low_freq_keywords)

### Lemmatize keywords

In [18]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
lemm = WordNetLemmatizer()

def lemmatize_keywords(keywords):
    return [lemm.lemmatize(kw) for kw in keywords]

high_quality_df['plot_keyword'] = high_quality_df['plot_keyword'].apply(lemmatize_keywords)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
high_quality_df.describe().T

,count,mean,std,min,25%,50%,75%,max
rating,5849.0,6.935886,0.608144,6.0,6.4,6.9,7.4,9.0
votes,5849.0,105878.269790,154516.265224,10000.0,20000.0,45000.0,121000.0,2000000.0
Action,5849.0,0.242435,0.428592,0.0,0.0,0.0,0.0,1.0
Adventure,5849.0,0.169431,0.375164,0.0,0.0,0.0,0.0,1.0
Animation,5849.0,0.043255,0.203449,0.0,0.0,0.0,0.0,1.0
Biography,5849.0,0.088904,0.284630,0.0,0.0,0.0,0.0,1.0
Comedy,5849.0,0.352197,0.477696,0.0,0.0,0.0,1.0,1.0
Crime,5849.0,0.227389,0.419182,0.0,0.0,0.0,0.0,1.0
Drama,5849.0,0.626261,0.483837,0.0,0.0,1.0,1.0,1.0
Family,5849.0,0.038126,0.191517,0.0,0.0,0.0,0.0,1.0


In [20]:
high_quality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5849 entries, 0 to 5848
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_title   5849 non-null   object 
 1   rating        5849 non-null   float64
 2   user_rating   5849 non-null   object 
 3   genres        5849 non-null   object 
 4   plot_keyword  5849 non-null   object 
 5   director      5849 non-null   object 
 6   top_5_casts   5849 non-null   object 
 7   writer        5849 non-null   object 
 8   votes         5849 non-null   int64  
 9   Action        5849 non-null   int32  
 10  Adventure     5849 non-null   int32  
 11  Animation     5849 non-null   int32  
 12  Biography     5849 non-null   int32  
 13  Comedy        5849 non-null   int32  
 14  Crime         5849 non-null   int32  
 15  Drama         5849 non-null   int32  
 16  Family        5849 non-null   int32  
 17  Fantasy       5849 non-null   int32  
 18  Film-Noir     5849 non-null 

In [21]:
high_quality_df.isnull().sum()

movie_title     0
rating          0
user_rating     0
genres          0
plot_keyword    0
director        0
top_5_casts     0
writer          0
votes           0
Action          0
Adventure       0
Animation       0
Biography       0
Comedy          0
Crime           0
Drama           0
Family          0
Fantasy         0
Film-Noir       0
History         0
Horror          0
Music           0
Musical         0
Mystery         0
Romance         0
Sci-Fi          0
Sport           0
Thriller        0
War             0
Western         0
dtype: int64

In [22]:
high_quality_df.to_csv('cleaned_high_quality_dataset.csv', index=False)